In [1]:
import geopandas as gpd
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import os

In [2]:
def shp2pgsql(file, engine):
    """单个shp文件入库"""
    file_name = os.path.split(file)[1]
    print('正在写入:'+file)
    tbl_name = file_name.split('.')[0]  # 表名
    map_data = gpd.GeoDataFrame.from_file(file)
    spatial_ref = map_data.crs.srs.split(':')[-1]  # 读取shp的空间参考
    map_data['geometry'] = map_data['geometry'].apply(
        lambda x: WKTElement(x.wkt, spatial_ref))
    # geopandas 的to_sql()方法继承自pandas, 将GeoDataFrame中的数据写入数据库
    map_data.to_sql(
        name=tbl_name,
        con=engine,
        if_exists='replace', # 如果表存在，则替换原有表
        chunksize=1000,  # 设置一次入库大小，防止数据量太大卡顿
        # 指定geometry的类型,这里直接指定geometry_type='GEOMETRY'，防止MultiPolygon无法写入
        dtype={'geometry': Geometry(
            geometry_type='GEOMETRY', srid=spatial_ref)},
        method='multi'
    )
    return None

In [3]:
def shp2pgsql_batch(dir_name, username, password, hostname, dbname):
    """创建批量任务"""
    os.chdir(dir_name)  # 改变当前工作目录到指定路径
    engine = create_engine('postgresql+psycopg2://{}:{}@{}/{}'.format(username, password, hostname, dbname))
    file_list = os.listdir(dir_name)
    for file in file_list:
        if file.split('.')[-1] == 'shp':
            file = os.path.abspath(file)
            shp2pgsql(file, engine)
    return None

In [5]:
# 执行任务计划
if __name__ == '__main__':
    file_path = r'/Users/creative/Desktop/data'
    username = 'postgres'
    password = '843211'
    hostname = 'localhost'
    dbname = '基础地理数据库'
    shp2pgsql_batch(file_path, username, password, hostname,dbname)

正在写入:/Users/creative/Desktop/data/国界线.shp


ArgumentError: srid must be convertible to an integer